In [1]:
"""
Code to download ERA5 reanalysis data at 6hr intervals as netcdfs using the cds api.

Select variables then provide inputs before running.
Make sure target directory exists before downloading.

***  Make sure username is changed otherwise you'll overwrite my files I've downloaded :) ***

For more info see:
https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview
https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-pressure-levels?tab=overview

or https://confluence.ecmwf.int/display/CKB/How+to+download+ERA5

Look here for active requests:
https://cds.climate.copernicus.eu/cdsapp#!/yourrequests?tab=form
"""

import cdsapi
import os

In [19]:
# SELECT VARIABLES (Only select ones on same model level together)

# SINGLE LEVEL
variable_names = []
#variable_names.append('mean_sea_level_pressure')
variable_names.append('10m_u_component_of_wind')
variable_names.append('10m_v_component_of_wind')
variable_names.append('sea_ice_cover')
# variable_names.append('total_cloud_cover')
#variable_names.append('surface_latent_heat_flux')
#variable_names.append('surface_net_solar_radiation')
#variable_names.append('surface_net_thermal_radiation')
#variable_names.append('surface_sensible_heat_flux')
#variable_names.append('surface_solar_radiation_downwards')
#variable_names.append('surface_thermal_radiation_downwards')
#variable_names.append('low_cloud_cover')
#variable_names.append('high_cloud_cover')
#variable_names.append('medium_cloud_cover')

# variable_names.append('2m_temperature')
variable_names.append('significant_height_of_combined_wind_waves_and_swell')
variable_names.append('significant_height_of_wind_waves')
variable_names.append('significant_height_of_total_swell')
# variable_names.append('maximum_individual_wave_height')
variable_names.append('peak_wave_period')
variable_names.append('mean_period_of_wind_waves')
variable_names.append('mean_wave_direction')
# variable_names.append('ocean_surface_stress_equivalent_10m_neutral_wind_speed')
# variable_names.append('ocean_surface_stress_equivalent_10m_neutral_wind_direction')
product_source = 'reanalysis-era5-single-levels'

# PRESSURE LEVELS (Mostly on different levels so download one variable at a time)
#variable_names = ['geopotential']
#variable_names = ['vertical_velocity']
# variable_names = ['temperature']
# pressure_list = ['500'] #pressures in mb of levels to be downloaded
# product_source = 'reanalysis-era5-pressure-levels'

# POTENTIAL VORTICITY LEVEL
#have to go through different method. Use mars request
#https://confluence.ecmwf.int/display/CKB/How+to+download+ERA5

In [28]:
# PROVIDE INPUTS
product_type = 'reanalysis' #'reanalysis' or 'ensemble_spread'
start_year = 2017
end_year = 2019 # end_year is included in download
month_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'] # note formate of leading zero for single digit months.
# month_list = ['07', '08']
time_list = ['00:00', '06:00', '12:00', '18:00']
# Area: North, West (-ve), South (-ve), East. Default: global
area_list  = [90, -180, 60, 180] 
# Latitude/longitude grid: east-west (longitude) and north-south resolution (latitude).
# Default: 0.25 x 0.25 for reanalysis, 0.5 for ocean waves. Might be coarser for ensemble members/spread
## see https://confluence.ecmwf.int/display/CKB/How+to+download+ERA5
# can also use 1.0, 1.0
grid_list =  [0.5, 0.5]
username = 'vcooper' #change to your own

In [29]:
# Get year list
year_list = []
for i in range(start_year, end_year+1):
    year_list.append(str(i))

In [30]:
# Get day list
day_list = []
for i in range(1, 31+1):
    day_list.append(str(i).zfill(2))

In [31]:
# Put inputs into dictionary
cdict = {
        'product_type' : product_type,
        'year': year_list,
        'day': day_list,
        'area' : area_list,
        'grid' : grid_list,
        'time' : time_list,
        'format' : 'netcdf',
        }

if product_source == 'reanalysis-era5-pressure-levels':
    cdict['pressure_level'] = pressure_list

In [32]:
for variable_name in variable_names:
    print(variable_name)

10m_u_component_of_wind
10m_v_component_of_wind
sea_ice_cover
significant_height_of_combined_wind_waves_and_swell
significant_height_of_wind_waves
significant_height_of_total_swell
peak_wave_period
mean_period_of_wind_waves
mean_wave_direction


In [26]:
## NOTE: FIX THIS NEXT TIME SO IT DOESNT OUTPUT MULTIPLE YEARS IN ONE FILE GROUPED BY MONTH
## it is hard to concatenate in current form
# Download data for each variable for each month
for variable_name in variable_names:
    cdict['variable'] = variable_name
    for month in month_list:
        cdict['month'] = month
        
        # Set file name
        mstr = 'months=' + month
        ystr = '_years=' + str(start_year) + 'to' + str(end_year)
        astr = '_minlat=' + str(area_list[2])
        gstr = '_grid=' + str(grid_list[0]) + 'x' + str(grid_list[1])

        if product_source == 'reanalysis-era5-single-levels':
            f_name = mstr  + ystr + astr + gstr + variable_name + '.nc'
        elif product_source == 'reanalysis-era5-pressure-levels':
            if len(pressure_list) == 1:
                pstr = '_plevel=' + pressure_list[0] + 'hPa'
            else:
                pstr = '_plevel=many'
            f_name = mstr  + ystr + astr + pstr + gstr + variable_name + '.nc'

        if product_type == 'reanalysis':
#             target = os.path.join('/glade', 'scratch', username, 'ERA5', 'native', f_name)
            target = os.path.join('/glade/campaign/univ', 'uwas0070', username, 'waveice','era5/native', f_name)
        elif product_type == 'ensemble_spread':
            target = os.path.join('/home', 'disk', 'sipn', username, 'ERA5', 'native', 'e_spread', f_name)
        print(target)
        c = cdsapi.Client()
        c.retrieve(product_source, cdict, target)

/glade/campaign/univ/uwas0070/vcooper/waveice/era5/native/months=01_years=2014to2016_minlat=60_grid=0.5x0.510m_u_component_of_wind.nc


2021-04-06 15:23:37,740 INFO Welcome to the CDS
2021-04-06 15:23:37,741 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2021-04-06 15:23:38,264 INFO Request is completed
2021-04-06 15:23:38,265 INFO Downloading https://download-0008.copernicus-climate.eu/cache-compute-0008/cache/data7/adaptor.mars.internal-1617744006.0166593-31521-3-8ea98fbe-095b-4ea8-960b-40508e94525e.nc to /glade/campaign/univ/uwas0070/vcooper/waveice/era5/native/months=01_years=2014to2016_minlat=60_grid=0.5x0.510m_u_component_of_wind.nc (31.2M)
2021-04-06 15:23:41,318 INFO Download rate 10.2M/s  


/glade/campaign/univ/uwas0070/vcooper/waveice/era5/native/months=02_years=2014to2016_minlat=60_grid=0.5x0.510m_u_component_of_wind.nc


2021-04-06 15:23:41,923 INFO Welcome to the CDS
2021-04-06 15:23:41,924 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2021-04-06 15:23:42,452 INFO Request is queued
2021-04-06 15:23:43,573 INFO Request is running
2021-04-06 15:24:15,493 INFO Request is completed
2021-04-06 15:24:15,494 INFO Downloading https://download-0001.copernicus-climate.eu/cache-compute-0001/cache/data5/adaptor.mars.internal-1617744223.3313093-9843-1-a7f67ee2-448e-46d2-868c-6579223bd8a3.nc to /glade/campaign/univ/uwas0070/vcooper/waveice/era5/native/months=02_years=2014to2016_minlat=60_grid=0.5x0.510m_u_component_of_wind.nc (28.5M)
2021-04-06 15:24:18,770 INFO Download rate 8.7M/s   


/glade/campaign/univ/uwas0070/vcooper/waveice/era5/native/months=03_years=2014to2016_minlat=60_grid=0.5x0.510m_u_component_of_wind.nc


...

2021-04-06 16:51:28,117 INFO Welcome to the CDS
2021-04-06 16:51:28,117 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2021-04-06 16:51:28,671 INFO Request is queued
2021-04-06 16:51:29,798 INFO Request is running
2021-04-06 16:51:50,405 INFO Request is completed
2021-04-06 16:51:50,406 INFO Downloading https://download-0007.copernicus-climate.eu/cache-compute-0007/cache/data3/adaptor.mars.internal-1617749489.0603325-1931-6-9382e973-6e74-4ee3-baeb-33d045541aac.nc to /glade/campaign/univ/uwas0070/vcooper/waveice/era5/native/months=10_years=2014to2016_minlat=60_grid=0.5x0.5mean_wave_direction.nc (31.2M)
2021-04-06 16:51:58,243 INFO Download rate 4M/s     


/glade/campaign/univ/uwas0070/vcooper/waveice/era5/native/months=11_years=2014to2016_minlat=60_grid=0.5x0.5mean_wave_direction.nc


2021-04-06 16:51:58,936 INFO Welcome to the CDS
2021-04-06 16:51:58,937 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2021-04-06 16:51:59,494 INFO Request is queued
2021-04-06 16:52:00,614 INFO Request is running
2021-04-06 16:52:21,012 INFO Request is completed
2021-04-06 16:52:21,013 INFO Downloading https://download-0000.copernicus-climate.eu/cache-compute-0000/cache/data7/adaptor.mars.internal-1617749519.6948516-18011-7-aa1bd865-2ff6-4977-a5e0-1b393abaaa25.nc to /glade/campaign/univ/uwas0070/vcooper/waveice/era5/native/months=11_years=2014to2016_minlat=60_grid=0.5x0.5mean_wave_direction.nc (30.2M)
2021-04-06 16:52:25,391 INFO Download rate 6.9M/s   


/glade/campaign/univ/uwas0070/vcooper/waveice/era5/native/months=12_years=2014to2016_minlat=60_grid=0.5x0.5mean_wave_direction.nc


2021-04-06 16:52:26,007 INFO Welcome to the CDS
2021-04-06 16:52:26,008 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2021-04-06 16:52:26,524 INFO Request is queued
2021-04-06 16:52:27,646 INFO Request is running
2021-04-06 16:52:59,568 INFO Request is completed
2021-04-06 16:52:59,569 INFO Downloading https://download-0002.copernicus-climate.eu/cache-compute-0002/cache/data0/adaptor.mars.internal-1617749546.7587144-16273-7-2f0fe1d5-ae50-48a3-8c32-c656f30259be.nc to /glade/campaign/univ/uwas0070/vcooper/waveice/era5/native/months=12_years=2014to2016_minlat=60_grid=0.5x0.5mean_wave_direction.nc (31.2M)
2021-04-06 16:53:06,455 INFO Download rate 4.5M/s   


In [27]:
#Print done?
print('Done')

Done
